In [1]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')

import py_entitymatching as em
import pandas as pd
import os,sys
import math

In [2]:
##Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]
pandas version: 0.19.2
magellan version: 0.1.0


In [3]:
## Reading csv tables into pandas dataframe and set the key attribute in the dataframe

A=em.read_csv_metadata('../Data/csv/yelp_list.csv')
A['ID'] = range(0,len(A))
em.set_key(A,'ID')


B=em.read_csv_metadata('../Data/csv/zomato_list.csv')
B['ID'] = range(0,len(B))
em.set_key(B,'ID')

No handlers could be found for logger "py_entitymatching.io.parsers"


True

In [4]:
##Data Cleaning1: Converting  price range to an absolute number. Will be applied for yelp list 


def clean_price_range( price_str ):

    avg_price = ''
    
    if not isinstance(price_str,basestring)  and  math.isnan(price_str):
        return price_str

    str2 = price_str.replace('$','')
        
    if '-' in str2:
        num_l = int(str2.split('-')[0])
        num_r = int(str2.split('-')[1])
        num_avg = (num_l+num_r)/2
        avg_price = '$' + str(num_avg)

    else:
        str_word = str2.split()[0].lower()
        num_price  = int(str2.split()[1])
        
        if str_word == 'above':
            num_avg = int(num_price + num_price/2)
            avg_price = '$' + str(num_avg)

        else:
            num_avg = num_price/2
            avg_price = '$' + str(num_avg)
            
    return avg_price

In [5]:
##Data Cleaning2: Removing "reviews" string from num_reviews column. Will be applied for zomato list


def clean_num_reviews(review_string) :
      return int(review_string.split()[0])

In [6]:
##Data Cleaning3: Removing "restaurant" if it is the last word in restaurant name. to prevent inconsistentcy.
## Will be applied for both lists


def clean_name( name_str):

    name_lower = name_str.lower()

    name_2 = name_lower.split()[-1]
    
    if name_2 == 'restaurant':
        name_lower = name_lower.replace('restaurant','')
        
    return name_lower

In [7]:
##Data Cleaning4: Removing quotes and shortening some words in address.Will be applied for zomato list 

def clean_address( address ):
    addr_2 = address.replace('\"','')
    addr_3 = addr_2.replace(',','')
    if "Street" in addr_3:
        addr_3 = addr_3.replace('Street','St')
    if "Boulevard" in addr_3:
        addr_3 = addr_3.replace('Boulevard','Blvd')
        
    return addr_3

In [8]:
## Data Cleaning5: Convert rating value into string type
def clean_rating_value(ratingValue):
    return str(ratingValue)

In [9]:
## Apply Data Cleaning to tables

A['price_range'] = A['price_range'].apply( clean_price_range )
A['name'] = A['name'].apply( clean_name )
A['ratingValue'] = A['ratingValue'].apply(clean_rating_value)



B['name'] = B['name'].apply( clean_name )
B['address'] = B['address'].apply( clean_address )
B['number_of_reviews'] = B['number_of_reviews'].apply( clean_num_reviews )
B['ratingValue'] = B['ratingValue'].apply(clean_rating_value)

In [10]:
#load candidate set data from previous session
C4 = em.load_object('./C4_410.pkl')

## Load C4 into em catalog
em.set_fk_ltable(C4, 'ltable_ID')
em.set_fk_rtable(C4, 'rtable_ID')
em.set_key(C4, '_id')
em.set_ltable(C4, A)
em.set_rtable(C4, B)

True

In [11]:
# Load labelled data 
G = em.load_object('./G_4thApril.pkl')
## Loading G into em catalog

em.set_fk_ltable(G, 'ltable_ID')
em.set_fk_rtable(G, 'rtable_ID')
em.set_key(G, '_id')
em.set_ltable(G, A)
em.set_rtable(G, B)


True

In [12]:
# Split G into development set (I) and evaluation set (J)
train_test = em.split_train_test(G, train_proportion=0.7)
I = train_test['train']
J = train_test['test']

In [13]:
## Best Matcher from previous stage
rf = em.RFMatcher()

In [14]:
match_f = em.get_features_for_matching(A, B)
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,number_of_reviews_number_of_reviews_exm,number_of_reviews,number_of_reviews,None,None,exact_match,<function number_of_reviews_number_of_reviews_exm at 0x7f812b866320>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,number_of_reviews_number_of_reviews_anm,number_of_reviews,number_of_reviews,None,None,abs_norm,<function number_of_reviews_number_of_reviews_anm at 0x7f812b822500>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,number_of_reviews_number_of_reviews_lev_dist,number_of_reviews,number_of_reviews,None,None,lev_dist,<function number_of_reviews_number_of_reviews_lev_dist at 0x7f812b874de8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,number_of_reviews_number_of_reviews_lev_sim,number_of_reviews,number_of_reviews,None,None,lev_sim,<function number_of_reviews_number_of_reviews_lev_sim at 0x7f81285bc6e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,price_range_price_range_lev_dist,price_range,price_range,None,None,lev_dist,<function price_range_price_range_lev_dist at 0x7f81285a5230>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,price_range_price_range_lev_sim,price_range,price_range,None,None,lev_sim,<function price_range_price_range_lev_sim at 0x7f81285a5848>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,price_range_price_range_jar,price_range,price_range,None,None,jaro,<function price_range_price_range_jar at 0x7f81285a5aa0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,price_range_price_range_jwn,price_range,price_range,None,None,jaro_winkler,<function price_range_price_range_jwn at 0x7f81285a50c8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,price_range_price_range_exm,price_range,price_range,None,None,exact_match,<function price_range_price_range_exm at 0x7f81285a5f50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,price_range_price_range_jac_qgm_3_qgm_3,price_range,price_range,qgm_3,qgm_3,jaccard,<function price_range_price_range_jac_qgm_3_qgm_3 at 0x7f81285a5de8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [15]:
## Drop unwanted features
match_f.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,29,30,31,32], inplace = True)
# List the names of the features generated
match_f['feature_name']

16              name_name_jac_qgm_3_qgm_3
17          name_name_cos_dlm_dc0_dlm_dc0
18          name_name_jac_dlm_dc0_dlm_dc0
19                          name_name_mel
20                     name_name_lev_dist
21                      name_name_lev_sim
22                          name_name_nmw
23                           name_name_sw
24        address_address_jac_qgm_3_qgm_3
25    address_address_cos_dlm_dc0_dlm_dc0
26                    address_address_mel
27               address_address_lev_dist
28                address_address_lev_sim
Name: feature_name, dtype: object

In [16]:
# Convert the I,J into a set of feature vectors using F

H = em.extract_feature_vecs(I, feature_table=match_f, attrs_after=['gold_labels'])
L= em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['gold_labels'])

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 

In [17]:
### Change from stage3- used only most relevant features to train the classifier

# Train random forest using feature vectors from I
rf.fit(table = H,
        exclude_attrs=['_id','ltable_ID','rtable_ID','gold_labels'],
        target_attr='gold_labels')

In [19]:
# Evaluate Random Forest
predictions_rf = rf.predict(table = L,
                        exclude_attrs=['_id','ltable_ID','rtable_ID','gold_labels'],
                        append = True, target_attr='predicted', inplace=False)

In [20]:
eval_result = em.eval_matches(predictions_rf, 'gold_labels', 'predicted')
em.print_eval_summary(eval_result)

Precision : 96.49% (55/57)
Recall : 96.49% (55/57)
F1 : 96.49%
False positives : 2 (out of 57 positive predictions)
False negatives : 2 (out of 49 negative predictions)


In [21]:
predictions_rf


,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,name_name_sw,address_address_jac_qgm_3_qgm_3,address_address_cos_dlm_dc0_dlm_dc0,address_address_mel,address_address_lev_dist,address_address_lev_sim,gold_labels,predicted
249,249,376,49,0.127660,0.258199,0.142857,0.591646,21.0,0.300000,-2.0,9.0,0.878049,0.857143,0.967068,3.0,0.921053,1,0
470,470,691,1784,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,13.0,13.0,0.714286,0.714286,0.930900,7.0,0.815789,1,1
97,97,165,843,0.666667,0.500000,0.333333,0.969231,1.0,0.923077,12.0,12.0,0.603774,0.721688,0.861862,17.0,0.653061,1,1
838,838,1220,2558,0.000000,0.000000,0.000000,0.524603,12.0,0.142857,0.0,2.0,0.852941,0.833333,0.988889,1.0,0.966667,0,0
1792,1792,2816,1451,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,25.0,25.0,0.698113,0.782624,0.932018,10.0,0.791667,1,1
1038,1038,1556,3235,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,21.0,21.0,1.000000,1.000000,1.000000,0.0,1.000000,1,1
1536,1536,2443,1117,0.000000,0.000000,0.000000,0.505476,20.0,0.200000,-6.0,3.0,0.818182,0.857143,0.980952,1.0,0.964286,0,0
277,277,407,296,0.000000,0.000000,0.000000,0.506944,13.0,0.187500,-4.0,3.0,0.809524,0.888889,0.984211,2.0,0.947368,0,0
363,363,532,2302,0.555556,0.666667,0.500000,0.895664,3.0,0.769231,8.0,8.0,1.000000,1.000000,1.000000,0.0,1.000000,1,1
882,882,1282,1202,0.000000,0.000000,0.000000,0.472222,12.0,0.200000,-2.0,2.0,0.825000,0.925820,0.941723,5.0,0.864865,0,0


In [22]:
## Convert Candidate set C4 to features
C4_features = em.extract_feature_vecs(C4, feature_table=match_f)


0%                          100%
[##############################] | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:05


In [23]:
C4_features

,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,name_name_sw,address_address_jac_qgm_3_qgm_3,address_address_cos_dlm_dc0_dlm_dc0,address_address_mel,address_address_lev_dist,address_address_lev_sim
0,0,0,29,0.050000,0.000000,0.000000,0.567100,9.0,0.181818,-2.0,2.0,1.000000,1.000000,1.000000,0.0,1.000000
1,1,1,67,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,6.0,6.0,0.717391,0.714286,0.936442,7.0,0.829268
2,2,6,1009,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,4.0,4.0,0.857143,0.833333,0.960560,3.0,0.906250
3,3,7,433,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,20.0,20.0,0.868421,0.833333,0.964107,3.0,0.914286
4,4,9,1128,0.444444,0.816497,0.666667,0.914286,6.0,0.571429,2.0,8.0,1.000000,1.000000,1.000000,0.0,1.000000
5,5,9,3115,0.000000,0.000000,0.000000,0.521429,16.0,0.200000,-2.0,2.0,0.823529,0.833333,0.981609,1.0,0.965517
6,6,10,2180,0.000000,0.000000,0.000000,0.411255,10.0,0.090909,-3.0,1.0,0.818182,0.833333,0.976190,1.0,0.964286
7,7,10,2762,0.040000,0.000000,0.000000,0.575758,9.0,0.181818,2.0,2.0,0.818182,0.833333,0.976190,1.0,0.964286
8,8,11,753,0.000000,0.000000,0.000000,0.562963,12.0,0.200000,-3.0,2.0,0.875000,0.857143,0.960254,3.0,0.918919
9,9,19,2296,0.038462,0.000000,0.000000,0.422222,12.0,0.200000,-4.0,3.0,1.000000,1.000000,1.000000,0.0,1.000000


In [27]:
# Apply random forest on candidate set features
predictions_rf_C4 = rf.predict(table = C4_features,
                        exclude_attrs=['_id','ltable_ID','rtable_ID'],
                        append = True, target_attr='predicted', inplace=False)

In [28]:
predictions_rf_C4


,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,name_name_sw,address_address_jac_qgm_3_qgm_3,address_address_cos_dlm_dc0_dlm_dc0,address_address_mel,address_address_lev_dist,address_address_lev_sim,predicted
0,0,0,29,0.050000,0.000000,0.000000,0.567100,9.0,0.181818,-2.0,2.0,1.000000,1.000000,1.000000,0.0,1.000000,1
1,1,1,67,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,6.0,6.0,0.717391,0.714286,0.936442,7.0,0.829268,1
2,2,6,1009,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,4.0,4.0,0.857143,0.833333,0.960560,3.0,0.906250,1
3,3,7,433,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,20.0,20.0,0.868421,0.833333,0.964107,3.0,0.914286,1
4,4,9,1128,0.444444,0.816497,0.666667,0.914286,6.0,0.571429,2.0,8.0,1.000000,1.000000,1.000000,0.0,1.000000,1
5,5,9,3115,0.000000,0.000000,0.000000,0.521429,16.0,0.200000,-2.0,2.0,0.823529,0.833333,0.981609,1.0,0.965517,0
6,6,10,2180,0.000000,0.000000,0.000000,0.411255,10.0,0.090909,-3.0,1.0,0.818182,0.833333,0.976190,1.0,0.964286,0
7,7,10,2762,0.040000,0.000000,0.000000,0.575758,9.0,0.181818,2.0,2.0,0.818182,0.833333,0.976190,1.0,0.964286,0
8,8,11,753,0.000000,0.000000,0.000000,0.562963,12.0,0.200000,-3.0,2.0,0.875000,0.857143,0.960254,3.0,0.918919,0
9,9,19,2296,0.038462,0.000000,0.000000,0.422222,12.0,0.200000,-4.0,3.0,1.000000,1.000000,1.000000,0.0,1.000000,0


In [30]:
em.save_object(predictions_rf_C4,'./predictions_rf_C4.pkl')
em.to_csv_metadata(predictions_rf_C4,'./predictions_rf_C4.csv')

True

In [ ]:
## Read C4 csv and predictions_rf_C4, look at "predicted" column of predictions table and merge tuple pairs of in each row
## of candidate set

## Create a new pandas frame with identical schema as the existing two tables

